In [1]:
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt 
%matplotlib inline 

/home/harrygong/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
np.random.seed(0)
tf.set_random_seed(0)

In [3]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
n_samples = mnist.train.num_examples

ModuleNotFoundError: No module named 'input_data'

In [5]:
def xavier_init(fan_in, fan_out, constant=1):
    low = -constant*np.sqrt(6.0/(fan_in + fan_out))
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), minval=low, maxval=high, dtype=tf.float32)

In [ ]:
class VariationalAutoencoder(object):
    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus, learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
        # Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        
        # Create autoencoder network 
        self._create_network()
        
        # Loss function and optimizer 
        self._create_loss_optimizer()
        
        # Inintialize variables 
        init = tf.global_variables_initializer()
        
        # Launch a session 
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
        
    def _create_network(self):
        network_weights = self._initialize_weights(**self.network_architecture)
        
        # Use recognition network to determine mean and (log) variance of Gaussian distribution in latent space
        self.z_mean, self.z_log_sigma_sq = self._recognition_network(network_weights["weights_recog"],
                                                                    network_weights["biases_recog"])
        
        # Draw one sample z from Gaussian distribution 
        n_z = self.network_architecture["n_z"]
        